## x.1 使用矩阵运算加速运算

这一章节主要讲解pytorch的张量带来的矢量化计算，致使计算速度提高

In [1]:

%matplotlib inline
import math
import time
import numpy as np
import torch
from d2l import torch as d2l


n = 10000
a = torch.ones(n)
b = torch.ones(n)



我们比较如果不矢量化计算，即把元素一个一个取出来，逐个相加

矢量化计算，则直接使用加号，不去遍历，速度结果差很多

In [2]:
# 不使用矩阵加速
c = torch.zeros(n)
t = time.time()
for i in range(n):
    c[i] = a[i] + b[i]
f'{time.time() - t:.5f} sec'

'0.26416 sec'

In [3]:
# 使用矩阵加速
t = time.time()
d = a + b
f'{time.time() - t:.5f} sec'

'0.00089 sec'

除此之外，本章还有数学知识，正态分布和平方损失MSE